In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import precision_score, mean_absolute_percentage_error,recall_score, accuracy_score,f1_score, confusion_matrix, mean_squared_error,ConfusionMatrixDisplay, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
pd.set_option('display.max_columns', None)
import folium
import pgeocode

## Part 1 - Exploratory Data Analysis

In [ ]:
house_data_df = pd.read_csv('processed_data_transformed.csv')
house_data_df.head()

In [ ]:
house_data_df.drop(['Unnamed: 0','Index','ID'], axis = 1, inplace=True)
house_data_df.head()

In [ ]:
house_data_df.describe()

In [ ]:
# Check for Missing Data
sns.heatmap(house_data_df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
# Check for the zero values of price
sum(house_data_df['Price'] == 0)

In [ ]:
# Check for the zero values of floor space
sum(house_data_df['Floor Space'] == 0)

In [ ]:
# Check for the class names of categorical data
house_data_df['Condition'].unique()

In [ ]:
house_data_df['Property Type'].unique()

In [ ]:
sns.boxplot(data=house_data_df, x='Bed', y='Property Type')

In [ ]:
sns.boxplot(data=house_data_df, x='Price', y='Property Type')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
sns.boxplot(data=house_data_df, y='Zip_Code', x='Price', orient="h", ax = ax)

In [ ]:
sns.pairplot(data=house_data_df[['Price', 'Bed', 'Bath', 'Floor Space']])

In [ ]:
sns.heatmap(data=house_data_df[['Price', 'Bed', 'Bath', 'Floor Space']].corr(), linewidths=0.5, center=0, annot=True)

## Part 2 - Data Processing

In [ ]:
# Drop the duplicates based on the address
house_data_df.drop_duplicates(subset = ['Address'], inplace = True)
house_data_df.shape

In [ ]:
# Remove the zero values om bathroom, price and floor space
house_data_df.drop(house_data_df[house_data_df.Price  == 0].index, inplace = True)
house_data_df.drop(house_data_df[house_data_df.Bath  == 0].index, inplace = True)
house_data_df.drop(house_data_df[house_data_df['Floor Space']  == 0].index, inplace = True)
house_data_df.shape

In [ ]:
house_data_df.describe()

In [ ]:
# Calculate the age of the property
house_data_df.drop(house_data_df[house_data_df['Years Sold']  == 'None'].index, inplace = True)
year_sold = [float(d) for d in house_data_df['Years Sold']]
house_data_df['Prop_Age'] = year_sold - house_data_df['Year Built']
house_data_df.head()

In [ ]:
# The distribution of numeric variables
house_data_df[['Price', 'Bed', 'Bath', 'Floor Space','Prop_Age','Num of Rooms']].hist(figsize = (8,6))
plt.show()

## Part 3 - Modeling

### Linear Regression Model 1 (Price as dependent variable)

In [ ]:
house_data_df1 = house_data_df
house_data_df1.drop(house_data_df1[house_data_df1.Price  < 300000].index, inplace = True)
house_data_df1.shape

In [ ]:
house_data_df_model1 = house_data_df1[['Price','Bed','Bath','Floor Space','Parking-Garage','Parking-Open','Zip_Code']]

In [ ]:
zip_dummies = pd.get_dummies(house_data_df_model1['Zip_Code'])
zip_dummies.head()

In [ ]:
df_model1 = pd.concat([house_data_df_model1, zip_dummies], axis=1)      
df_model1.drop(['Zip_Code', 94965.0], inplace=True, axis=1)
df_model1.head()

Dropped 94965.0 zip code from columns, it'd be the base 

In [ ]:
df_model1.columns = df_model1.columns.astype(str)
features = df_model1.drop(['Price'], axis = 1, inplace = False)
features = sm.add_constant(features)
price = df_model1['Price']
X_train, X_test, y_train, y_test = train_test_split(features, price, test_size=0.2, random_state=42)

In [ ]:
model_lr1 = sm.OLS(y_train, X_train).fit()
print(model_lr1.summary())

In [ ]:
mean_absolute_percentage_error(y_test, model_lr1.predict(X_test))

### Linear Regression Model 2 (Price as dependent variable, adding Num of Rooms, Condition, and Views to the independent variables)

In [ ]:
house_data_df2 = house_data_df
house_data_df2.drop(house_data_df2[house_data_df2.Price  < 10000].index, inplace = True) 
house_data_df2.head()

In [ ]:
# Manipulate the View
house_data_df2.loc[house_data_df2['Views'] == '{}', 'Views'] = 'No View'
house_data_df2.head()

In [ ]:
# Manipulate the Parking condition
house_data_df2['Parking'] = np.where(house_data_df2['Parking-Carport'] + house_data_df2['Parking-Garage'] + house_data_df2['Parking-Open'] >0 ,1,0)
house_data_df2.info()

In [ ]:
house_data_df_model2 = house_data_df2[['Price','Floor Space', 'Num of Rooms', 'Bed','Bath', 'Prop_Age', 'Condition', 'Views', 'Zip_Code', 'Parking']]

In [ ]:
# Dummies for zip code
zip_dummies = pd.get_dummies(house_data_df_model2['Zip_Code'])
df_model2 = pd.concat([house_data_df_model2, zip_dummies], axis=1)      
df_model2.drop(['Zip_Code', 94965.0], inplace=True, axis=1)

#Dummies for condition 
condition_dummies = pd.get_dummies(house_data_df_model2['Condition'])
df_model2 = pd.concat([df_model2, condition_dummies], axis=1)      
df_model2.drop(['Condition', 'Existing'], inplace=True, axis=1)

# Dummines for Views
view_dummies = pd.get_dummies(house_data_df_model2['Views'])
df_model2 = pd.concat([df_model2, view_dummies], axis=1)      
df_model2.drop(['Views', 'No View'], inplace=True, axis=1)

In [ ]:
df_model2.info()

In [ ]:
df_model2 = df_model2.dropna()

In [ ]:
df_model2.columns = df_model2.columns.astype(str)
features = df_model2.drop(['Price'], axis = 1, inplace = False)
features = sm.add_constant(features)
price = df_model2['Price']
X_train, X_test, y_train, y_test = train_test_split(features, price, test_size=0.2, random_state=123)

In [ ]:
model_lr2 = sm.OLS(y_train, X_train).fit()
print(model_lr2.summary())

In [ ]:
mean_absolute_percentage_error(y_test, model_lr2.predict(X_test))

### Linear Regression Model 3 (Price/sqft as dependent variable)

In [ ]:
house_data_df3 = house_data_df
# Calculate the price per sqft
house_data_df3['Price_per_sqft'] = house_data_df3.Price / house_data_df3['Floor Space']
house_data_df3.describe()

In [ ]:
# Detect the outliers based on IQR
Q1 = np.percentile(house_data_df3['Price_per_sqft'], 25, interpolation = 'midpoint')
Q3 = np.percentile(house_data_df3['Price_per_sqft'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1
upper=Q3+1.5*IQR
lower=Q1-1.5*IQR
print(upper, " - ", lower)

In [ ]:
# Remove the lower outliers
house_data_df3 = house_data_df3.drop(house_data_df3[house_data_df3['Price_per_sqft']  > 1728.0236971454897].index)
house_data_df3 = house_data_df3.drop(house_data_df3[house_data_df3['Price_per_sqft']  < 170.2150344115821].index)
house_data_df3.shape

In [ ]:
house_data_df_model3 = house_data_df3[['Price_per_sqft','Bed','Bath','Floor Space','Parking-Garage','Parking-Open','Property Type','Zip_Code','Prop_Age']]

In [ ]:
house_data_df_model3 = house_data_df_model3.dropna()
house_data_df_model3.shape

In [ ]:
type_dummies = pd.get_dummies(house_data_df_model3['Property Type'])
zip_dummies = pd.get_dummies(house_data_df_model3['Zip_Code'])
df_model3 = pd.concat([house_data_df_model3, type_dummies, zip_dummies], axis=1)  
df_model3.drop(['Property Type', 'Apartment'], inplace=True, axis=1)
df_model3.drop(['Zip_Code', 94965.0], inplace=True, axis=1)
df_model3.head()

In [ ]:
df_model3.columns = df_model3.columns.astype(str)
features = df_model3.drop(['Price_per_sqft'], axis = 1, inplace = False)
features = sm.add_constant(features)
price = df_model3['Price_per_sqft']
X_train, X_test, y_train, y_test = train_test_split(features, price, test_size=0.2, random_state=42)

In [ ]:
model_lr3 = sm.OLS(y_train, X_train).fit()
print(model_lr3.summary())

In [ ]:
mean_absolute_percentage_error(y_test, model_lr3.predict(X_test))

### RF Regressor (Price/sqft as dependent variable)

In [ ]:
model_rf = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
mean_absolute_percentage_error(y_test, model_rf.predict(X_test))

### GBRegressor (Price/sqft as dependent variable)

In [ ]:
model_gbr = GradientBoostingRegressor(random_state=0).fit(X_train, y_train)
mean_absolute_percentage_error(y_test, model_gbr.predict(X_test))

### Neural Networks (Price/sqft as dependent variable)

In [ ]:
modelnn = Sequential()
modelnn.add(Dense(20,activation='relu'))
modelnn.add(Dense(20,activation='relu'))
modelnn.add(Dense(20,activation='relu'))
modelnn.add(Dense(20,activation='relu'))
modelnn.add(Dense(1)) 
modelnn.compile(optimizer='adam',loss='mse')

In [ ]:
modelnn.fit(x=X_train,y=y_train, validation_data=(X_test, y_test), batch_size=128, epochs=400)

In [ ]:
mean_absolute_percentage_error(y_test, modelnn.predict(X_test))

### Neural Networks - Design 2 (Price/sqft as dependent variable)

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
NN_model.fit(x=X_train,y=y_train, validation_data=(X_test, y_test), batch_size=128, epochs=400)

In [ ]:
mean_absolute_percentage_error(y_test, NN_model.predict(X_test))

## Part 4 - Result Visualization

In [ ]:
def get_coef_table(lin_reg):
    ''' lin_reg is a fitted statsmodels regression model
    Return a dataframe containing coefficients, pvalues, and the confidence intervals
    '''
    err_series = lin_reg.params - lin_reg.conf_int()[0]
    coef_df = pd.DataFrame({'coef': lin_reg.params.values[0:],
                            'ci_err': err_series.values[0:],
                            'pvalue': lin_reg.pvalues.round(4).values[0:],
                            'varname': err_series.index.values[0:]
                           })
    return coef_df

In [ ]:
get_coef_table(model_lr3)

In [ ]:
df_result = get_coef_table(model_lr3)
df_result['coef_diff'] =  df_result['coef'][0] + df_result['coef']
df_result = df_result[['varname','coef_diff']]
df_result = df_result[14:]
df_result['coef_diff'] = df_result['coef_diff'] - np.min(df_result['coef_diff'])
df_result['varname'] = df_result['varname'].astype(float).astype(int)
df_result

In [ ]:
nomi = pgeocode.Nominatim('us')

edf = df_result.copy()
edf['varname'] = edf['varname'].astype(float).astype(int).astype(str)
edf['Latitude'] = (nomi.query_postal_code(edf['varname'].tolist()).latitude)
edf['Longitude'] = (nomi.query_postal_code(edf['varname'].tolist()).longitude)

edf

In [ ]:
df_viz = edf.dropna()

from folium.plugins import MarkerCluster
map = folium.Map(location=[37.7749, -122.4194], zoom_start=11)
marker_cluster = MarkerCluster().add_to(map) 

# add a marker for each zipcode
for index, row in df_viz.iterrows():
    zipcode = row['varname']
    lat = row['Latitude']
    lon = row['Longitude']
    price_premium = row['coef_diff']
    tooltip = f"Zipcode: {zipcode}"
    marker = folium.Marker(location=[lat, lon], tooltip=tooltip, popup="""
                  <i>Price Premium: </i> <br> <b>${}</b> 
                  """.format(round(price_premium)))
    marker.add_to(marker_cluster)

In [ ]:
print(map)